In [131]:
from lexmachina import LexMachinaClient
from lexmachina import StateCaseQueryRequest

from typing import Optional


In [132]:
client = LexMachinaClient(config_file_path='../config/config.ini')

In order to get the judges for a specific party, here we first locate the party id.
Here we are using a very simplistic method where we take the most frequently occurring party returned from party search to be the correct one. 
That can be revisited, or else you can look up the party in the LexMachina product ahead of time.

In [91]:
def num_lex_cases_for_party(party_id: int): 
    query = StateCaseQueryRequest()
    query.include_parties(party_id)
    query.include_state('CA')
    cases = client.query_state_cases_case(query=query, options={'pageThrough': True}, page_size=100)
    return len(cases)

def get_party_id(name: str) -> int:
    candidates = client.search_parties(name, page_size=10)['parties']
    candidates = [party['partyId'] for party in candidates]
    num_cases = [num_lex_cases_for_party(cand) for cand in candidates]
    most_freq = sorted(zip(candidates, num_cases), key=lambda tup: tup[1], reverse=True)[0]
    return most_freq[0]

In [119]:
party_id = get_party_id('liberty mutual')
print(party_id)

39480


Next, we'll gather information about the cases this party was in, filtering by role if desired.
From the cases, we'll aggregate the judges presiding over these cases, ultimately selecting those who did so the most often.

In [139]:
def lex_judges_for_party(party_id: int, role: Optional[str] = None, state: str = 'CA'):
    cases = list(lex_cases_for_party_role(party_id, role, state))
    print(f'there are {len(cases)} cases in {state} where {party_id} has the role: {role}')
    for i, case_info in enumerate(cases):
        if i % 20 == 0:
            print(f'case {i} of {len(cases)}')
        yield from judges_for_lex_case(case_info['stateCaseId'])
        
def lex_cases_for_party_role(party_id: int, role: Optional[str] = None, state: str = 'CA'):
    query = StateCaseQueryRequest()
    query.include_state(state)
    if not role:
        query.include_parties(party_id)
    elif role.lower() == 'defendant':
        query.parties_include_defendant(party_id)
    elif role.lower() == 'plaintiff':
        query.parties_include_plaintiff(party_id)
    return client.query_state_cases_case(query=query, options={'pageThrough': True}, page_size=100)

def judges_for_lex_case(case_id: int):
    lex_case = client.get_state_cases(case_id)
    judges = lex_case['judges']
    yield from judges


In [ ]:
from collections import Counter

judge_counts = Counter()
judge_id_to_name = dict()
all_judges = lex_judges_for_party(party_id=party_id, role='defendant')
for i, judge_info in enumerate(all_judges):
    judge_name = judge_info['name']
    judge_id = judge_info['stateJudgeId']
    judge_id_to_name[judge_id] = judge_name
    judge_counts[judge_id] += 1   

there are 628 cases in CA where 39480 has the role: defendant
case 0 of 628
case 20 of 628
case 40 of 628
case 60 of 628
case 80 of 628
case 100 of 628


In [ ]:
judge_counts.most_common(10)